# Store Sales Demand Forecast

### Introduction

The dataset used in this code was obtained from [Store Sales - Time Series Forecasting](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data?select=train.csv). Due to github filesize limit, dataset not checked into the repo. Download to local first.  

* Starts with forecasting sales for ONE family with holidays factor: bread and bakery. 
* Add oil regressor
* Add another onpromotion regressor
* Expand to multiple families forecast 

In [1]:
# !pip install prophet
# !pip install duckdb
# !pip install seaborn

### Import libraries

In [2]:
import numpy as np
import pandas as pd
import duckdb
import math
import itertools 

#matplotlib libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors
import seaborn as sns

#date libraries
from dateutil import parser
from datetime import datetime, timedelta, date

#prophet library
from prophet import Prophet
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import cross_validation

#pandas options
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#matplotlib setting defaults
sns.set(font="Arial",
        rc={
 "axes.axisbelow": False,
 #"axes.edgecolor": "lightgrey",
 #"axes.facecolor": "None",
 "axes.grid": False,
 #"axes.labelcolor": "dimgrey",
 "axes.spines.right": False,
 "axes.spines.top": False,
 #"figure.facecolor": "white",
 "lines.solid_capstyle": "round",
 #"patch.edgecolor": "w",
 #"patch.force_edgecolor": True,
 #"text.color": "dimgrey",
 "xtick.bottom": False,
 #"xtick.color": "dimgrey",
 "xtick.direction": "out",
 "xtick.top": False,
 #"ytick.color": "dimgrey",
 "ytick.direction": "out",
 "ytick.left": False,
 "ytick.right": False})

Importing plotly failed. Interactive plots will not work.


### Function: Check for null values

In [3]:
def missing_data(input_data):
    total = input_data.isnull().sum()
    percent = (input_data.isnull().sum()/input_data.isnull().count()*100)
    table = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
    types = []
    for col in input_data.columns: 
        dtype = str(input_data[col].dtype)
        types.append(dtype)
    table["Types"] = types
    return(pd.DataFrame(table))

### Function: Check Model Accuracy (Mean Absolute Percentage Error (MAPE))

In [4]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

## Data Prep

In [5]:
df = pd.read_csv('data/train.csv')
df.head
df.onpromotion.unique()

array([  0,   3,   5,   1,  56,  20,  19,   2,   4,  18,  17,  12,   6,
         7,  10,   9,  50,   8,  16,  42,  51,  13,  15,  47,  21,  40,
        37,  54,  24,  58,  22,  59,  11,  45,  25,  55,  26,  43,  35,
        14,  28,  46,  36,  32,  53,  57,  27,  39,  41,  30,  29,  49,
        23,  48,  44,  38,  31,  52,  33,  34,  61,  60, 116,  86,  73,
       113, 102,  68, 104,  93,  70,  92, 121,  72, 178, 174, 161, 118,
       105, 172, 163, 167, 142, 154, 133, 180, 181, 173, 165, 168, 186,
       140, 149, 145, 169, 188,  62,  84, 111,  65, 107,  63, 101,  87,
       125,  94, 114, 171, 153, 170, 166, 141, 155, 179, 192, 131, 147,
       151, 189,  79,  74, 110,  64,  67,  99, 123, 157, 117, 150, 182,
       162, 160, 194, 135, 190,  69, 108,  89, 126, 156, 103, 146, 132,
       177, 164, 176, 112,  75, 109,  91, 128, 175, 187, 148, 137, 184,
       196, 144, 158, 119, 106,  66, 100,  90, 120, 115,  98, 159, 152,
       185, 139, 143,  80, 124,  71, 134, 193,  78,  88, 122, 13

In the demo, ignore store_nbr factor, assume all sales come from one store. However, you can build models for each individual store. 
* filter out store_nbr 1 sales: df[df['store_nbt'] == 1], 
* check how many unique stores: df.store_nbr.unique()

In [6]:
##-- check data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


### Assigning the approprate data types

In [7]:
##-- standardize column names
df.columns = df.columns.str.replace(' ', '_').str.lower()

##--convert date to datetime
df['date'] = pd.to_datetime(df['date'], format= "%Y-%m-%d")

In [9]:
##-- check minimum and maximum date values
(min_date, max_date) = duckdb.query("select min(date) as min_date, max(date) as max_date from df").fetchone()
(min_date, max_date)

(datetime.datetime(2013, 1, 1, 0, 0), datetime.datetime(2017, 8, 15, 0, 0))

###  Aggregate the data to the family and sales daily levels
* This will provide daily demand data for each product family

In [10]:
sql = """
select date, family, sum(sales) sales, sum(onpromotion) onpromotion
from df
group by 1,2
order by 2,1
"""
agg_df = duckdb.query(sql).fetchdf()
agg_df.head(10)

,date,family,sales,onpromotion
0,2013-01-01,AUTOMOTIVE,0.00,0.00
1,2013-01-02,AUTOMOTIVE,255.00,0.00
2,2013-01-03,AUTOMOTIVE,161.00,0.00
3,2013-01-04,AUTOMOTIVE,169.00,0.00
4,2013-01-05,AUTOMOTIVE,342.00,0.00
5,2013-01-06,AUTOMOTIVE,360.00,0.00
6,2013-01-07,AUTOMOTIVE,189.00,0.00
7,2013-01-08,AUTOMOTIVE,229.00,0.00
8,2013-01-09,AUTOMOTIVE,164.00,0.00
9,2013-01-10,AUTOMOTIVE,164.00,0.00


In [ ]:
##-- check the data
agg_df.info()

### creating a pivot table for the aggregated data

In [11]:
total_sales_df = agg_df.pivot(index='date',columns='family', values='sales')
total_sales_df.head()

family,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,EGGS,FROZEN FOODS,GROCERY I,GROCERY II,HARDWARE,HOME AND KITCHEN I,HOME AND KITCHEN II,HOME APPLIANCES,HOME CARE,LADIESWEAR,LAWN AND GARDEN,LINGERIE,"LIQUOR,WINE,BEER",MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.00,0.00,2.00,810.00,0.00,180.59,0.00,186.00,143.00,71.09,46.00,29.65,700.00,15.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,5.00,105.00,0.00,110.80,25.00,0.00,0.00,42.64,37.85,0.00,0.00,0.00
2013-01-02,255.00,0.00,207.00,72092.00,0.00,26246.32,0.00,74629.00,23381.00,15754.50,10932.00,7115.39,202020.00,1476.00,46.00,0.00,0.00,21.00,0.00,0.00,121.00,469.00,2411.00,0.00,20871.46,17204.00,0.00,0.00,13975.88,5338.11,0.00,0.00,1526.75
2013-01-03,161.00,0.00,125.00,52105.00,0.00,18456.48,0.00,55893.00,18001.00,11172.46,7358.00,4760.81,144878.00,1048.00,37.00,0.00,0.00,15.00,0.00,0.00,83.00,366.00,2476.00,0.00,16597.40,12568.00,0.00,0.00,10674.39,3591.39,0.00,0.00,1094.31
2013-01-04,169.00,0.00,133.00,54167.00,0.00,16721.97,0.00,52064.00,18148.00,10143.21,6760.00,4525.93,135754.00,1031.00,57.00,0.00,0.00,13.00,0.00,0.00,127.00,382.00,4796.00,0.00,21625.96,11303.00,0.00,0.00,10772.52,4472.97,0.00,0.00,1293.12
2013-01-05,342.00,0.00,191.00,77818.00,0.00,22367.76,0.00,70128.00,23082.00,13734.95,8576.00,5781.61,188356.00,1273.00,87.00,0.00,0.00,11.00,0.00,0.00,180.00,458.00,6715.00,0.00,20879.09,16819.00,0.00,0.00,13475.01,5830.07,0.00,0.00,1245.64


In [ ]:
##-- visualizing the data

for column in total_sales_df.columns:
    plt.plot(total_sales_df[column], color='black', alpha=0.7)
    plt.title(column)
    plt.show()

### Data Cleaning:

* Check for missing values
* Eliminate low sales volume products that cannot be accurately predicted.
* Remove products with insufficient historical data. Visual inspection or counting daily '0' sales can identify these.
* Remove outliers using z-score calculations. This will eliminate outliers close to 0 sales for all product categories.
* Filter data to only include 08/15/15 - 08/15/17 date range. Check if additional data needs removal after this filter. If data looks clean, proceed with modeling pipeline.

In [ ]:
##-- checking missing data
missing_data(total_sales_df)

In [12]:
##-- only keep products with avg daily sales greater than $1000

# get all columns
all_columns = total_sales_df.columns

#Keep products with avg daily sales greater than $1000
keep_columns = total_sales_df.columns[total_sales_df.apply(np.mean,axis='rows') >= 1000]

# remove products with avg daily sales less than $1000
exclusion_columns =  all_columns.difference(keep_columns)

print("Dropping "+ str(len(exclusion_columns))+" columns due to insufficient data volume.")
total_sales_df = total_sales_df[keep_columns]
total_sales_df.head()

Dropping 15 columns due to insufficient data volume.


family,BEVERAGES,BREAD/BAKERY,CLEANING,DAIRY,DELI,EGGS,FROZEN FOODS,GROCERY I,GROCERY II,HOME AND KITCHEN I,HOME CARE,"LIQUOR,WINE,BEER",MEATS,PERSONAL CARE,POULTRY,PREPARED FOODS,PRODUCE,SEAFOOD
date,,,,,,,,,,,,,,,,,,
2013-01-01,810.00,180.59,186.00,143.00,71.09,46.00,29.65,700.00,15.00,0.00,0.00,105.00,110.80,25.00,42.64,37.85,0.00,0.00
2013-01-02,72092.00,26246.32,74629.00,23381.00,15754.50,10932.00,7115.39,202020.00,1476.00,0.00,0.00,2411.00,20871.46,17204.00,13975.88,5338.11,0.00,1526.75
2013-01-03,52105.00,18456.48,55893.00,18001.00,11172.46,7358.00,4760.81,144878.00,1048.00,0.00,0.00,2476.00,16597.40,12568.00,10674.39,3591.39,0.00,1094.31
2013-01-04,54167.00,16721.97,52064.00,18148.00,10143.21,6760.00,4525.93,135754.00,1031.00,0.00,0.00,4796.00,21625.96,11303.00,10772.52,4472.97,0.00,1293.12
2013-01-05,77818.00,22367.76,70128.00,23082.00,13734.95,8576.00,5781.61,188356.00,1273.00,0.00,0.00,6715.00,20879.09,16819.00,13475.01,5830.07,0.00,1245.64


In [ ]:
##-- Filter data to only include 08/15/15 - 08/15/17 date range
total_sales_df = total_sales_df[total_sales_df.index>='2015-08-15']

### Eliminate low sales volume products that cannot be accurately predicted.

* Group categories by data volume, as higher volume improves forecasting accuracy
* Higher volume results in less noise and lower error bars
* Grouping allows visualizing forecasts on similar scales for comparison

### Category Grouping

In [13]:
#lets break it down by thirds for low, mid, high
avg_daily_sales = total_sales_df.apply(np.mean, axis='rows').sort_values()
low, mid = np.percentile(avg_daily_sales, [33,66])

In [ ]:
avg_daily_sales, low, mid

In [ ]:
low_vol_columns = list(avg_daily_sales[avg_daily_sales<=low].index)
mid_vol_columns = avg_daily_sales[(avg_daily_sales>low) & (avg_daily_sales<mid)].index
high_vol_columns = avg_daily_sales[avg_daily_sales>=mid].index
low_vol_columns,mid_vol_columns,high_vol_columns

In [ ]:
#total_sales_df[low_vol_columns].plot.line()
plt.plot(total_sales_df[low_vol_columns])
plt.legend(low_vol_columns, loc='best', bbox_to_anchor=(1.1, 1.1))
plt.xticks(rotation=30)
plt.show()

In [ ]:
plt.plot(total_sales_df[mid_vol_columns])
plt.legend(mid_vol_columns, loc='best', bbox_to_anchor=(1.1, 1.1))
plt.xticks(rotation=30)
plt.show()

In [ ]:
plt.plot(total_sales_df[high_vol_columns])
plt.legend(high_vol_columns, loc='best', bbox_to_anchor=(1.1, 1.1))
plt.xticks(rotation=30)
plt.show()

We observe here that bread/bakery is one of the items with a high volume metric. We will focus on forecasting sales for this particular item to see how it performs.

### Testing with "Bread/Bakery"

In [14]:
#setting variables
feature = 'BREAD/BAKERY'
prediction_days = 30

In [15]:
df = total_sales_df[[feature]].reset_index()
df.head()

family,date,BREAD/BAKERY
0,2013-01-01,180.59
1,2013-01-02,26246.32
2,2013-01-03,18456.48
3,2013-01-04,16721.97
4,2013-01-05,22367.76


#### holiday info

In [16]:
# read holidays data
holidays_df = pd.read_csv('data/holidays_events.csv')

# only keep national holidays
national_holidays = holidays_df[(holidays_df['locale'] == 'National') & (holidays_df['type'] == 'Holiday')]
national_holidays['lower_window'] = -2
national_holidays['upper_window'] = 1
holidays = national_holidays[['date','description','lower_window', 'upper_window']].rename(columns={'date': 'ds', 'description': 'holiday'})
holidays

/var/folders/37/ynryj5js4px7jpw5mhyc9pgr0000gn/T/ipykernel_4013/1693635373.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  national_holidays['lower_window'] = -2
/var/folders/37/ynryj5js4px7jpw5mhyc9pgr0000gn/T/ipykernel_4013/1693635373.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  national_holidays['upper_window'] = 1


,ds,holiday,lower_window,upper_window
14,2012-08-10,Primer Grito de Independencia,-2,1
19,2012-10-09,Independencia de Guayaquil,-2,1
21,2012-11-02,Dia de Difuntos,-2,1
22,2012-11-03,Independencia de Cuenca,-2,1
37,2012-12-25,Navidad,-2,1
41,2013-01-01,Primer dia del ano,-2,1
44,2013-02-11,Carnaval,-2,1
45,2013-02-12,Carnaval,-2,1
51,2013-04-29,Viernes Santo,-2,1
52,2013-05-01,Dia del Trabajo,-2,1


https://pypi.org/project/holidays/

In [17]:
#Variables

# forecast_start_dt = date(2017,7,1) #data set ends on (2017,8,15)
forecast_start_date = pd.Timestamp('2017-07-01')

### Predicting sales with Facebook Prophet Model

In [18]:
#PROPHET MODEL

df_copy = df.copy()
df_copy = df_copy.rename(columns={'date': 'ds', feature: 'y'})

# Convert ds to datetime and y to numberic if not done yet
# df_copy['ds'] = pd.to_datetime(df_copy['ds'])
# df_copy[['y']] = df_copy[['y']].apply(pd.to_numeric)

train_set = df_copy[(df_copy['ds'] < forecast_start_date) ]

m = Prophet(holidays=holidays)

# if there is no holidays input, add standard holidays by country
# m.add_country_holidays(country_name='US')

m.fit(train_set)

future = m.make_future_dataframe(periods=prediction_days)
fcst_prophet_train = m.predict(future)

#adding filter to only add the forecasted data into predicted_df
filter = fcst_prophet_train['ds']>=forecast_start_date 
predicted_df = fcst_prophet_train[filter][['ds','yhat']]

23:09:16 - cmdstanpy - INFO - Chain [1] start processing
23:09:16 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:
##-- checking predicted values
predicted_df.head(10)

<b> variable considerations for prophet model: </b> growth, changepoint_prior_scale, changepoint_range, yearly/weekly/daily seasonality, seasonality mode, holidays

In [19]:
##-- comparing true and predicted values
df_copy = df.copy()
df_copy.columns = ['ds', 'ytrue']
predicted_df = predicted_df.merge(df_copy)

predicted_df

,ds,yhat,ytrue
0,2017-07-01,34359.41,38304.17
1,2017-07-02,39455.00,44839.79
2,2017-07-03,30485.88,32696.44
3,2017-07-04,27951.39,28806.21
4,2017-07-05,28428.95,28769.54
5,2017-07-06,25847.56,24758.21
6,2017-07-07,27740.43,28049.34
7,2017-07-08,34219.63,32995.15
8,2017-07-09,39315.19,40633.64
9,2017-07-10,30349.74,29713.90


In [20]:
##-- checking accuracy
100-mape(predicted_df['ytrue'], predicted_df['yhat'])

93.23824005494791

### Cross Validation with Prophet
Find the "best" parameters by testing over various periods of time with those parameters (cross-validation).

In [115]:
m = Prophet(holidays=holidays)
m.fit(train_set)
df_cv = cross_validation(m, initial='365 days', period='30 days', horizon = '30 days')

21:07:23 - cmdstanpy - INFO - Chain [1] start processing
21:07:23 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/10 [00:00<?, ?it/s]

21:07:23 - cmdstanpy - INFO - Chain [1] start processing
21:07:23 - cmdstanpy - INFO - Chain [1] done processing
21:07:23 - cmdstanpy - INFO - Chain [1] start processing
21:07:23 - cmdstanpy - INFO - Chain [1] done processing
21:07:24 - cmdstanpy - INFO - Chain [1] start processing
21:07:24 - cmdstanpy - INFO - Chain [1] done processing
21:07:24 - cmdstanpy - INFO - Chain [1] start processing
21:07:24 - cmdstanpy - INFO - Chain [1] done processing
21:07:24 - cmdstanpy - INFO - Chain [1] start processing
21:07:25 - cmdstanpy - INFO - Chain [1] done processing
21:07:25 - cmdstanpy - INFO - Chain [1] start processing
21:07:25 - cmdstanpy - INFO - Chain [1] done processing
21:07:25 - cmdstanpy - INFO - Chain [1] start processing
21:07:25 - cmdstanpy - INFO - Chain [1] done processing
21:07:25 - cmdstanpy - INFO - Chain [1] start processing
21:07:26 - cmdstanpy - INFO - Chain [1] done processing
21:07:26 - cmdstanpy - INFO - Chain [1] start processing
21:07:26 - cmdstanpy - INFO - Chain [1]

In [116]:
#total_sales_df[total_sales_df['PRODUCE']<10000]

In [117]:
df_p = performance_metrics(df_cv)
df_p

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,3 days,19584234.86,4425.41,3210.68,0.10,0.08,0.10,0.61
1,4 days,12775852.69,3574.33,2825.60,0.09,0.08,0.09,0.63
2,5 days,7867837.01,2804.97,2253.03,0.07,0.06,0.08,0.70
3,6 days,6729542.34,2594.14,2020.68,0.06,0.05,0.07,0.71
4,7 days,4176227.37,2043.58,1593.45,0.05,0.04,0.05,0.83
5,8 days,4480687.03,2116.76,1690.95,0.05,0.05,0.05,0.80
6,9 days,4281088.89,2069.08,1626.47,0.06,0.05,0.05,0.87
7,10 days,4757505.95,2181.17,1726.56,0.06,0.06,0.06,0.87
8,11 days,6175850.13,2485.13,1883.60,0.07,0.05,0.06,0.87
9,12 days,5698090.23,2387.07,1861.32,0.06,0.06,0.06,0.87


df_p gives you the overall MAPE, but if you want to get more granular and calculate the daily differences using the mape metric, the code is below. This allows you to detect issues in predicting certain time periods, which are inherent issues in the data that you may or may not be able to fix. Timeseries is the prime example that garbage in is going to be garbage out.

In [118]:
df_cv['mape'] = (df_cv['y']-df_cv['yhat'])/(df_cv['y'])*100
df_cv['overestimate'] = df_cv['yhat'] > df_cv['y'] 

df_cv.sort_values('mape',ascending=False).head(10)

,ds,yhat,yhat_lower,yhat_upper,y,cutoff,mape,overestimate
58,2016-11-01,15341.89,12768.71,18130.59,26266.84,2016-10-03,41.59,False
119,2017-01-02,31138.48,28124.01,34022.97,45621.16,2017-01-01,31.75,False
120,2017-01-03,25335.83,22458.74,28172.51,35916.70,2017-01-01,29.46,False
208,2017-04-01,35326.07,32342.84,38565.86,45561.27,2017-03-02,22.46,False
59,2016-11-02,23317.10,20557.35,25956.42,29341.76,2016-10-03,20.53,False
121,2017-01-04,25626.89,22733.35,28347.40,32084.67,2017-01-01,20.13,False
177,2017-03-01,29258.42,26349.69,32191.14,36097.44,2017-01-31,18.95,False
93,2016-12-06,25130.34,22453.85,27798.88,30523.18,2016-12-02,17.67,False
27,2016-10-01,31670.98,28850.28,34488.41,38323.72,2016-09-03,17.36,False
149,2017-02-01,27573.55,24763.14,30456.51,33187.20,2017-01-31,16.92,False


In [119]:
df_cv.head()

,ds,yhat,yhat_lower,yhat_upper,y,cutoff,mape,overestimate
0,2016-09-04,37554.25,34991.97,40160.81,41518.69,2016-09-03,9.55,False
1,2016-09-05,27773.88,25199.41,30291.54,31015.78,2016-09-03,10.45,False
2,2016-09-06,25332.87,22670.82,27942.44,28444.62,2016-09-03,10.94,False
3,2016-09-07,25693.79,22986.82,28644.93,26954.62,2016-09-03,4.68,False
4,2016-09-08,22639.47,20055.22,25214.81,22999.11,2016-09-03,1.56,False


#### Optimizing for "BREAD/BAKERY" feature. What that means is... <b> lets hypertune this model! </b>

In [120]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0]
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = [] 

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(train_set)  # Fit model with given params
    df_cv = cross_validation(m, initial='365 days', period='30 days', horizon = '30 days', parallel='processes')
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses

21:07:27 - cmdstanpy - INFO - Chain [1] start processing
21:07:27 - cmdstanpy - INFO - Chain [1] done processing
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
21:07:30 - cmdstanpy - INFO - Chain [1] start processing
21:07:30 - cmdstanpy - INFO - Chain [1] start processing
21:07:30 - cmdstanpy - INFO - Chain [1] start processing
21:07:30 - cmdstanpy - INFO - Chain [1] start processing
21:07:30 - cmdstanpy - INFO - Chain [1] done processing
21:07:30 - cmdstanpy - INFO - Chain [1] done processing
21:07:30 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
21:07:30 - cmdstanpy - INFO - Chain [1] done processing
21:07:30 - cmdstanpy - INFO - Chain [1] done processing
21:07:30 - cmdstanpy - INFO - Chain [1] start p

21:07:47 - cmdstanpy - INFO - Chain [1] start processing
21:07:47 - cmdstanpy - INFO - Chain [1] start processing
21:07:47 - cmdstanpy - INFO - Chain [1] done processing
21:07:47 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
21:07:47 - cmdstanpy - INFO - Chain [1] start processing
21:07:47 - cmdstanpy - INFO - Chain [1] start processing
21:07:47 - cmdstanpy - INFO - Chain [1] done processing
21:07:48 - cmdstanpy - INFO - Chain [1] done processing
21:07:48 - cmdstanpy - INFO - Chain [1] start processing
21:07:48 - cmdstanpy - INFO - Chain [1] done processing
21:07:48 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
21:07:48 - cmdstanpy - INFO - Chain [1] start processing
21:07:48 - cmdstanpy - INFO - Chain [1] start processing
21:07:48 - cmdstanpy - INFO - Chain [1] done processing
21:07:48 

21:08:09 - cmdstanpy - INFO - Chain [1] start processing
21:08:09 - cmdstanpy - INFO - Chain [1] start processing
21:08:09 - cmdstanpy - INFO - Chain [1] start processing
21:08:09 - cmdstanpy - INFO - Chain [1] start processing
21:08:09 - cmdstanpy - INFO - Chain [1] done processing
21:08:09 - cmdstanpy - INFO - Chain [1] done processing
21:08:09 - cmdstanpy - INFO - Chain [1] done processing
21:08:09 - cmdstanpy - INFO - Chain [1] done processing
21:08:09 - cmdstanpy - INFO - Chain [1] start processing
21:08:09 - cmdstanpy - INFO - Chain [1] start processing
21:08:09 - cmdstanpy - INFO - Chain [1] done processing
21:08:09 - cmdstanpy - INFO - Chain [1] done processing
21:08:10 - cmdstanpy - INFO - Chain [1] start processing
21:08:10 - cmdstanpy - INFO - Chain [1] done processing
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactiv

21:08:30 - cmdstanpy - INFO - Chain [1] start processing
21:08:30 - cmdstanpy - INFO - Chain [1] done processing
21:08:30 - cmdstanpy - INFO - Chain [1] done processing
21:08:30 - cmdstanpy - INFO - Chain [1] done processing
21:08:30 - cmdstanpy - INFO - Chain [1] done processing
21:08:31 - cmdstanpy - INFO - Chain [1] start processing
21:08:31 - cmdstanpy - INFO - Chain [1] start processing
21:08:31 - cmdstanpy - INFO - Chain [1] start processing
21:08:31 - cmdstanpy - INFO - Chain [1] start processing
21:08:31 - cmdstanpy - INFO - Chain [1] done processing
21:08:31 - cmdstanpy - INFO - Chain [1] done processing
21:08:31 - cmdstanpy - INFO - Chain [1] done processing
21:08:31 - cmdstanpy - INFO - Chain [1] done processing
21:08:31 - cmdstanpy - INFO - Chain [1] start processing
21:08:31 - cmdstanpy - INFO - Chain [1] start processing
21:08:31 - cmdstanpy - INFO - Chain [1] done processing
21:08:31 - cmdstanpy - INFO - Chain [1] done processing
21:08:31 - cmdstanpy - INFO - Chain [1] s

In [121]:
print(tuning_results)

    changepoint_prior_scale  seasonality_prior_scale    rmse
0                      0.00                     0.01 4051.89
1                      0.00                     0.10 3897.94
2                      0.00                     1.00 3904.44
3                      0.00                    10.00 3967.89
4                      0.01                     0.01 3905.24
5                      0.01                     0.10 3791.39
6                      0.01                     1.00 3778.77
7                      0.01                    10.00 3797.62
8                      0.10                     0.01 3999.48
9                      0.10                     0.10 3897.02
10                     0.10                     1.00 3899.65
11                     0.10                    10.00 3897.32
12                     0.50                     0.01 4036.47
13                     0.50                     0.10 3932.67
14                     0.50                     1.00 3931.58
15                     0

In [122]:
tuning_results.sort_values('rmse')

,changepoint_prior_scale,seasonality_prior_scale,rmse
6,0.01,1.00,3778.77
5,0.01,0.10,3791.39
7,0.01,10.00,3797.62
9,0.10,0.10,3897.02
11,0.10,10.00,3897.32
1,0.00,0.10,3897.94
10,0.10,1.00,3899.65
2,0.00,1.00,3904.44
4,0.01,0.01,3905.24
15,0.50,10.00,3928.37


In [123]:
tuning_results.sort_values('rmse').reset_index(drop=True).iloc[0]

changepoint_prior_scale      0.01
seasonality_prior_scale      1.00
rmse                      3778.77
Name: 0, dtype: float64

In [124]:
dict(tuning_results.sort_values('rmse').reset_index(drop=True).iloc[0])

{'changepoint_prior_scale': 0.01,
 'seasonality_prior_scale': 1.0,
 'rmse': 3778.7655261238397}

In [125]:
params_dictionary = dict(tuning_results.sort_values('rmse').reset_index(drop=True).drop('rmse',axis='columns').iloc[0])

m = Prophet(changepoint_prior_scale = params_dictionary['changepoint_prior_scale'], 
            seasonality_prior_scale = params_dictionary['seasonality_prior_scale'],
           holidays=holidays)


In [126]:
m.fit(train_set)

future = m.make_future_dataframe(periods=prediction_days)
fcst_prophet_train = m.predict(future)

#adding filter to only add the forecasted data into predicted_df
filter = fcst_prophet_train['ds']>=forecast_start_date 
predicted_df = fcst_prophet_train[filter][['ds','yhat']]

df_copy = df.copy()
df_copy.columns = ['ds', 'ytrue']
predicted_df = predicted_df.merge(df_copy)
mape(predicted_df['ytrue'], predicted_df['yhat'])

21:08:35 - cmdstanpy - INFO - Chain [1] start processing
21:08:36 - cmdstanpy - INFO - Chain [1] done processing


6.769814187779996

In [127]:
100-mape(predicted_df['ytrue'], predicted_df['yhat'])

93.23018581222

### Add additional regressor - Oil price

In [21]:
# still using df created for BREAD/BAKERY before 
df.head()

family,date,BREAD/BAKERY
0,2013-01-01,180.59
1,2013-01-02,26246.32
2,2013-01-03,18456.48
3,2013-01-04,16721.97
4,2013-01-05,22367.76


In [22]:
# read in oil price
oil_df = pd.read_csv('data/oil.csv')
# convert to datetime
oil_df['date'] = pd.to_datetime(oil_df['date'])
# rename columns
oil_df.columns=['date', 'oil_price']

In [23]:
sql = """
select df.date as ds, "BREAD/BAKERY" as y, oil_price from df
left join oil_df on df.date = oil_df.date
"""
sales_oil_df = duckdb.query(sql).fetchdf()
# Fill NaN values in the 'Sales' column with the previous non-null value
sales_oil_df['oil_price'].fillna(method='ffill', inplace=True)
sales_oil_df.head()

/var/folders/37/ynryj5js4px7jpw5mhyc9pgr0000gn/T/ipykernel_4013/3864427118.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sales_oil_df['oil_price'].fillna(method='ffill', inplace=True)


,ds,y,oil_price
0,2013-01-02,26246.32,93.14
1,2013-01-04,16721.97,93.12
2,2013-01-07,17646.15,93.20
3,2013-01-08,15805.76,93.21
4,2013-01-09,16002.83,93.08


In [24]:
#PROPHET MODEL

df_copy = sales_oil_df.copy()

train_set = df_copy[(df_copy['ds'] < forecast_start_date) ]

m = Prophet(holidays=holidays)
m.add_regressor('oil_price')
m.fit(train_set)

future = m.make_future_dataframe(periods=prediction_days)
future['oil_price'] = 48.81

fcst_prophet_train = m.predict(future)

#adding filter to only add the forecasted data into predicted_df
filter = fcst_prophet_train['ds']>=forecast_start_date 
predicted_df = fcst_prophet_train[filter][['ds','yhat']]
predicted_df.head()

23:10:03 - cmdstanpy - INFO - Chain [1] start processing
23:10:03 - cmdstanpy - INFO - Chain [1] done processing


,ds,yhat
1638,2017-07-01,34662.82
1639,2017-07-02,39744.16
1640,2017-07-03,30081.06
1641,2017-07-04,27551.47
1642,2017-07-05,28014.83


In [25]:
##-- comparing true and predicted values
df_copy = df.copy()
df_copy.columns = ['ds', 'ytrue']
predicted_df = predicted_df.merge(df_copy)

predicted_df

,ds,yhat,ytrue
0,2017-07-01,34662.82,38304.17
1,2017-07-02,39744.16,44839.79
2,2017-07-03,30081.06,32696.44
3,2017-07-04,27551.47,28806.21
4,2017-07-05,28014.83,28769.54
5,2017-07-06,25435.18,24758.21
6,2017-07-07,27327.65,28049.34
7,2017-07-08,34501.71,32995.15
8,2017-07-09,39586.29,40633.64
9,2017-07-10,29930.20,29713.90


In [26]:
##-- checking accuracy
100-mape(predicted_df['ytrue'], predicted_df['yhat'])

93.81606262133442

### Add another regressor - onpromotion

In [27]:
# onpromotion: how many products of that category were on promotion on that day
promotion_df = agg_df[agg_df['family'] == feature]
sql="""
select sales_oil_df.*, onpromotion
from sales_oil_df 
left join promotion_df on sales_oil_df.ds = promotion_df.date
"""
sale_oil_promotion_df = duckdb.query(sql).fetchdf()
sale_oil_promotion_df.head()

,ds,y,oil_price,onpromotion
0,2013-01-02,26246.32,93.14,0.00
1,2013-01-04,16721.97,93.12,0.00
2,2013-01-07,17646.15,93.20,0.00
3,2013-01-08,15805.76,93.21,0.00
4,2013-01-09,16002.83,93.08,0.00


In [28]:
future_on_promotion = sale_oil_promotion_df[(sale_oil_promotion_df['ds'] >= forecast_start_date)]
future_on_promotion.head()

,ds,y,oil_price,onpromotion
965,2017-07-03,32696.44,44.88,325.00
966,2017-07-05,28769.54,45.11,515.00
967,2017-07-06,24758.21,45.52,423.00
968,2017-07-07,28049.34,44.25,445.00
969,2017-07-10,29713.90,44.40,445.00


In [29]:
#PROPHET MODEL

df_copy = sale_oil_promotion_df.copy()

train_set = df_copy[(df_copy['ds'] < forecast_start_date) ]

m = Prophet(holidays=holidays)
m.add_regressor('oil_price')
m.add_regressor('onpromotion')
m.fit(train_set)

future = m.make_future_dataframe(periods=prediction_days, include_history=False)
future_sql = """
select future.ds, oil_price, onpromotion
from future
left join future_on_promotion on future.ds = future_on_promotion.ds
"""
future = duckdb.query(future_sql).fetchdf()
future

23:10:28 - cmdstanpy - INFO - Chain [1] start processing
23:10:29 - cmdstanpy - INFO - Chain [1] done processing


,ds,oil_price,onpromotion
0,2017-07-01,48.81,309.00
1,2017-07-02,48.81,1756.00
2,2017-07-03,44.88,325.00
3,2017-07-04,44.88,319.00
4,2017-07-05,45.11,515.00
5,2017-07-06,45.52,423.00
6,2017-07-07,44.25,445.00
7,2017-07-08,48.81,441.00
8,2017-07-09,48.81,1721.00
9,2017-07-10,44.40,445.00


In [30]:
fcst_prophet_train = m.predict(future)

# #adding filter to only add the forecasted data into predicted_df
# filter = fcst_prophet_train['ds']>=forecast_start_date 
predicted_df = fcst_prophet_train[['ds','yhat']]
predicted_df.head()

,ds,yhat
0,2017-07-01,34252.62
1,2017-07-02,40864.32
2,2017-07-03,29947.74
3,2017-07-04,27409.10
4,2017-07-05,28215.29


In [31]:
##-- comparing true and predicted values
df_copy = df.copy()
df_copy.columns = ['ds', 'ytrue']
predicted_df = predicted_df.merge(df_copy)

predicted_df

,ds,yhat,ytrue
0,2017-07-01,34252.62,38304.17
1,2017-07-02,40864.32,44839.79
2,2017-07-03,29947.74,32696.44
3,2017-07-04,27409.10,28806.21
4,2017-07-05,28215.29,28769.54
5,2017-07-06,25501.71,24758.21
6,2017-07-07,27414.32,28049.34
7,2017-07-08,34336.98,32995.15
8,2017-07-09,40662.68,40633.64
9,2017-07-10,30023.93,29713.90


In [32]:
##-- checking accuracy
100-mape(predicted_df['ytrue'], predicted_df['yhat'])

94.16976226725474